# This file will conduct all the pre processing to format our TLE and Ephemeris Data
We will first preprocess the SpaceX ephemeris data from spacetrack. All data will be SpaceX_Ephemeris_552_SpaceX_2025-03-23UTC05:21:05_(01-15)

# Import Dependencies

In [13]:
from pathlib import Path
import pandas as pd
import re
import dask.dataframe as dd
import psycopg2
from datetime import datetime, timedelta, timezone
from Raj_Database_credentials import db_host,db_port,db_password,db_user

## 1. Create a function to read the ITC format epoch

In [14]:
def parse_custom_datetime(dt_str):
    """
    Converts a string in yyyDOYhhmmss.sss format to a UTC datetime object.
    """
    # Regex for yyyDOYhhmmss.sss
    match = re.match(r'(\d{4})(\d{3})(\d{2})(\d{2})(\d{2})\.(\d+)', dt_str)
    
    if not match:
        print(f"Failed to parse datetime string: {dt_str}")
        return None
    
    year = int(match.group(1))
    day_of_year = int(match.group(2))
    hour = int(match.group(3))
    minute = int(match.group(4))
    second = int(match.group(5))
    microsecond = int(float(f"0.{match.group(6)}") * 1e6)  # convert fraction to microseconds
    
    # Create the date from year and day of year
    dt = datetime(year, 1, 1, tzinfo=timezone.utc) + timedelta(days=day_of_year - 1, 
                                                               hours=hour, 
                                                               minutes=minute, 
                                                               seconds=second, 
                                                               microseconds=microsecond)
    return dt

# 2. Read the Starlink Ephemeris data and populate a dictionary

In [15]:
import re
from pathlib import Path
from datetime import datetime, timedelta, timezone

# Your folder path
folder_path = Path("Starlink_Ephemeris_2025-03-23UTC05-21-05")

# Initialize your main data dictionary
main_data_dict = {
    'NORAD_ID': [],
    'NAME': [],
    'DATETIME': [],
    'POS_X': [],
    'POS_Y': [],
    'POS_Z': [],
    'VEL_X': [],
    'VEL_Y': [],
    'VEL_Z': []
}

def parse_custom_datetime(dt_str):
    """
    Converts a string in yyyDOYhhmmss.sss format to a UTC datetime object.
    """
    # Regex for yyyDOYhhmmss.sss
    match = re.match(r'(\d{4})(\d{3})(\d{2})(\d{2})(\d{2})\.(\d+)', dt_str)
    
    if not match:
        print(f"Failed to parse datetime string: {dt_str}")
        return None
    
    year = int(match.group(1))
    day_of_year = int(match.group(2))
    hour = int(match.group(3))
    minute = int(match.group(4))
    second = int(match.group(5))
    microsecond = int(float(f"0.{match.group(6)}") * 1e6)  # convert fractional seconds to microseconds
    
    # Build the UTC datetime
    dt = datetime(year, 1, 1, tzinfo=timezone.utc) + timedelta(
        days=day_of_year - 1,
        hours=hour,
        minutes=minute,
        seconds=second,
        microseconds=microsecond
    )
    
    return dt

# Loop through all .txt files in the folder
for file in folder_path.glob("*.txt"):
    print(f"Processing file: {file.name}")

    # Extract NORAD_ID and SAT_NAME from filename using regex
    NORAD_ID_match = re.search(r"MEME_(\d+)_STARLINK", file.name)
    SAT_NAME_match = re.search(r"STARLINK-(\d+)_", file.name)

    NORAD_ID = NORAD_ID_match.group(1) if NORAD_ID_match else "Unknown"
    SAT_NAME = SAT_NAME_match.group(1) if SAT_NAME_match else "Unknown"

    # Open the file and read it line by line
    with open(file, "r", encoding="utf-8") as f:
        for idx, line in enumerate(f, start=1):
            # Starts at line 5, then every 4 lines after
            if idx >= 5 and (idx - 5) % 4 == 0:
                clean_line = line.strip()
                values = clean_line.split()

                if len(values) >= 7:  # Safety check in case a line is incomplete
                    # Parse the datetime value into ISO 8601 format
                    dt_obj = parse_custom_datetime(values[0])
                    if dt_obj is None:
                        print(f"Skipping invalid datetime at line {idx} in file {file.name}")
                        continue
                    datetime_iso = dt_obj.isoformat()

                    # Extract position and velocity data
                    positionX = values[1]
                    positionY = values[2]
                    positionZ = values[3]
                    velocityX = values[4]
                    velocityY = values[5]
                    velocityZ = values[6]

                    # Append to dictionary lists
                    main_data_dict["NORAD_ID"].append(NORAD_ID)
                    main_data_dict["NAME"].append(SAT_NAME)
                    main_data_dict["DATETIME"].append(datetime_iso)
                    main_data_dict["POS_X"].append(positionX)
                    main_data_dict["POS_Y"].append(positionY)
                    main_data_dict["POS_Z"].append(positionZ)
                    main_data_dict["VEL_X"].append(velocityX)
                    main_data_dict["VEL_Y"].append(velocityY)
                    main_data_dict["VEL_Z"].append(velocityZ)

                else:
                    print(f"Skipping incomplete line {idx} in file {file.name}")

print("Finished making dictionary!")


Processing file: MEME_44714_STARLINK-1008_0820354_Operational_1426996500_UNCLASSIFIED.txt
Processing file: MEME_44716_STARLINK-1010_0820346_Operational_1426996020_UNCLASSIFIED.txt
Processing file: MEME_44717_STARLINK-1011_0820356_Operational_1426996620_UNCLASSIFIED.txt
Processing file: MEME_44718_STARLINK-1012_0820322_Operational_1426994580_UNCLASSIFIED.txt
Processing file: MEME_44719_STARLINK-1013_0820339_Operational_1426995600_UNCLASSIFIED.txt
Processing file: MEME_44720_STARLINK-1014_0820031_Operational_1426984320_UNCLASSIFIED.txt
Processing file: MEME_44721_STARLINK-1015_0820355_Operational_1426996560_UNCLASSIFIED.txt
Processing file: MEME_44723_STARLINK-1017_0820352_Operational_1426996380_UNCLASSIFIED.txt
Processing file: MEME_44724_STARLINK-1019_0820315_Operational_1426994160_UNCLASSIFIED.txt
Processing file: MEME_44725_STARLINK-1020_0820328_Operational_1426994940_UNCLASSIFIED.txt
Processing file: MEME_44726_STARLINK-1021_0820312_Operational_1426993980_UNCLASSIFIED.txt
Processing

## 2. Populate the Ephemeris DB in PostgreSQL

In [ ]:
import psycopg2

# Database connection details (you probably already have these)
conn = psycopg2.connect(
    host=db_host,
    dbname="Starlink Training Data",
    user=db_user,
    password=db_password,
    port=db_port
)

cur = conn.cursor()

# Prepare the data as a list of tuples for insertion
data_to_insert = []
for i in range(len(main_data_dict["NORAD_ID"])):
    # Prepare each row as a tuple
    row = (
        main_data_dict["NORAD_ID"][i],
        main_data_dict["NAME"][i],
        main_data_dict["DATETIME"][i],  
        float(main_data_dict["POS_X"][i]),
        float(main_data_dict["POS_Y"][i]),
        float(main_data_dict["POS_Z"][i]),
        float(main_data_dict["VEL_X"][i]),
        float(main_data_dict["VEL_Y"][i]),
        float(main_data_dict["VEL_Z"][i])
    )
    data_to_insert.append(row)

# SQL insert statement (excluding id because it's SERIAL)
insert_query = """
INSERT INTO Ephemeris
(NORAD_ID, NAME, DATETIME, POS_X, POS_Y, POS_Z, VEL_X, VEL_Y, VEL_Z)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

try:
    # Bulk insert all records
    cur.executemany(insert_query, data_to_insert)
    conn.commit()
    print(f"{cur.rowcount} rows inserted successfully into the Ephemeris table.")

except Exception as e:
    conn.rollback()
    print(f"Error inserting data: {e}")

finally:
    cur.close()
    conn.close()


28574773 rows inserted successfully into the Ephemeris table.


: 